In [1]:
import pandas as pd

In [2]:
train_data=pd.read_csv('train.csv', index_col='Id')
test_data=pd.read_csv('test.csv', index_col='Id')

Again, removing rows missing values in target var and then separating.

In [3]:
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y=train_data.SalePrice
train_data.drop(['SalePrice'], axis=1, inplace=True)

In [4]:
# select nly numeric columns

In [5]:
numeric_col=[col for col in train_data.columns if train_data[col].dtype in ['int64','float64']]

In [6]:
X=train_data[numeric_col].copy()

In [7]:
X_test=test_data[numeric_col].copy()

In [8]:
X.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,548,0,61,0,0,0,0,0,2,2008
2,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,460,298,0,0,0,0,0,0,5,2007
3,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,608,0,42,0,0,0,0,0,9,2008
4,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,642,0,35,272,0,0,0,0,2,2006
5,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,836,192,84,0,0,0,0,0,12,2008


Implementing pipeline - all in one execute

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline=Pipeline(steps=[
    ('preprocessor', SimpleImputer()),
    ('model', RandomForestRegressor(n_estimators=50, random_state=0))
])

In [10]:
from sklearn.model_selection import cross_val_score

scores=-1* cross_val_score(my_pipeline, X, y, cv=5, scoring='neg_mean_absolute_error')

In [11]:
print("Average MAE score:", scores.mean())

Average MAE score: 18276.410356164386


In [12]:
def get_scores(n_estimators) :
    my_pipeline=Pipeline(steps=[
    ('preprocessor', SimpleImputer()),
    ('model', RandomForestRegressor(n_estimators, random_state=0))
])
    scores=-1* cross_val_score(my_pipeline, X, y, cv=3, scoring='neg_mean_absolute_error')
    return scores.mean()

In [19]:
print(get_scores(100))

18395.2151680032


In [20]:
print(get_scores(200))

18248.345889801505


Function that scores for different values of n_estimators

In [21]:
results={}
for i in range(1,9) :
    results[50*i]= get_scores(50*i)

In [22]:
print(results)

{50: 18353.8393511688, 100: 18395.2151680032, 150: 18288.730020956387, 200: 18248.345889801505, 250: 18255.26922247291, 300: 18275.241922621914, 350: 18270.29183308043, 400: 18270.197974402367}


In [23]:
# find best model

n_estimator_best=min(results, key=results.get)

In [24]:
print(n_estimator_best)

200
